In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from dotenv import load_dotenv
import os

# Exemple de chargement des données depuis une base SQL ou CSV
# df = pd.read_sql_query(query, conn)  # Adapté à ta source SQL
# Connexion à PostgreSQL
conn = psycopg2.connect(
   host = os.getenv("DB3_HOST"),
port = os.getenv("DB3_PORT"),
database = os.getenv("DB3_NAME"),
username = os.getenv("DB3_USER"),
password = os.getenv("DB3_PASSWORD")
)

# Requête SQL pour récupérer les données
query = """
SELECT 
    dd.specialite, 
    dd.mention, 
    dd.type_diplome, 
    dc.sexe, 
    dc.resultat,
    dc.orientation,  -- Cible
    dc.code_pays, 
    fa.moyenne_bac, 
    fa.score
FROM "fact_Admission" fa
JOIN "Dim_candidat" dc ON fa.fk_candidat = dc.pk_candidat
JOIN "dim_diplome" dd ON fa.fk_diplome = dd.pk_diplome
WHERE dc.orientation IS NOT NULL;
"""

# Chargement des données
df = pd.read_sql_query(query, conn)
conn.close()


# Colonnes
categorical_cols = ['specialite', 'mention', 'type_diplome', 'sexe', 'resultat', 'code_pays']
num_cols = ['moyenne_bac', 'score']
target_col = 'orientation'  # La colonne cible que tu veux prédire

# Encodage des variables catégorielles
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(df[categorical_cols])

# Création d'un DataFrame avec les variables encodées
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))

# Ajout des variables numériques
encoded_df[num_cols] = df[num_cols].astype(float).values  # Assurer que les numériques sont des floats
X = encoded_df
y = df[target_col]

# Application de SMOTE pour le rééquilibrage des classes
smote = SMOTE()
X_res, y_res = smote.fit_resample(X, y)

# Split en données d'entraînement et de test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Entraînement du modèle RandomForest
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Évaluation du modèle
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Sauvegarde du modèle et de l'encodeur pour une utilisation future
joblib.dump(model, 'orientation_model.pkl')
joblib.dump(encoder, 'encoder.pkl')

# Fonction de prédiction
def predict_orientation(input_data):
    # Charger le modèle et l'encodeur sauvegardés
    model = joblib.load('orientation_model.pkl')
    encoder = joblib.load('encoder.pkl')

    # Forcer la conversion des données numériques (si elles sont fournies sous forme de chaînes)
    input_data['moyenne_bac'] = float(input_data['moyenne_bac'])
    input_data['score'] = float(input_data['score'])

    # Transformer l'input en DataFrame
    input_df = pd.DataFrame([input_data])

    # Encoder les variables catégorielles
    input_encoded = pd.DataFrame(
        encoder.transform(input_df[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols)
    )

    # Ajouter les variables numériques à la DataFrame encodée
    input_encoded[num_cols] = input_df[num_cols].values

    # Effectuer la prédiction
    prediction = model.predict(input_encoded)
    return prediction[0]

# Exemple de prédiction
# Exemple modifié pour tester une autre combinaison
sample_input = {
    'specialite': 'MATH',
    'mention': 'Assez Bien',
    'type_diplome': 'bac',
    'sexe': 'F',
    'resultat': 'LISTE ATTENTE',
    'code_pays': 'TN',
    'moyenne_bac': '14.61',  # Ici, il est fourni en chaîne, il sera converti en float
    'score': '85.76'         # Même conversion ici
}

# Affichage du résultat de la prédiction
prediction = predict_orientation(sample_input)
print(f"Prédiction : {prediction}")



C:\Users\felah\AppData\Local\Temp\ipykernel_21620\1761072027.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


              precision    recall  f1-score   support

     ELECTRO       0.82      0.77      0.79       121
     GENICIV       0.80      0.87      0.83       114
        INFO       0.74      0.79      0.76       126
     TELCOMM       0.74      0.66      0.70       111

    accuracy                           0.77       472
   macro avg       0.77      0.77      0.77       472
weighted avg       0.77      0.77      0.77       472

Prédiction : GENICIV


In [42]:
import psycopg2
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# 1. Connexion à PostgreSQL et récupération des données
conn = psycopg2.connect(
    host="localhost",
    database="ADMISSION_DW_NEW",
    user="postgres",
    password="rania"
)

# Requête SQL pour récupérer toutes les données nécessaires
query = """
SELECT 
    dd.specialite, 
    dd.mention, 
    dd.type_diplome, 
    dc.sexe, 
    dc.resultat,
    dc.orientation, 
    dc.code_pays, 
    fa.moyenne_bac, 
    fa.score
FROM "fact_Admission" fa
JOIN "Dim_candidat" dc ON fa.fk_candidat = dc.pk_candidat
JOIN "dim_diplome" dd ON fa.fk_diplome = dd.pk_diplome;
"""

# Chargement des données dans un DataFrame pandas
df = pd.read_sql_query(query, conn)
conn.close()

# 2. Préparation des données
# Créer un LabelEncoder pour chaque colonne catégorielle
le_specialite = LabelEncoder()
le_mention = LabelEncoder()
le_type_diplome = LabelEncoder()
le_sexe = LabelEncoder()
le_resultat = LabelEncoder()
le_orientation = LabelEncoder()
le_code_pays = LabelEncoder()

# Appliquer l'encodeur sur les colonnes catégorielles
df['specialite'] = le_specialite.fit_transform(df['specialite'])
df['mention'] = le_mention.fit_transform(df['mention'])
df['type_diplome'] = le_type_diplome.fit_transform(df['type_diplome'])
df['sexe'] = le_sexe.fit_transform(df['sexe'])
df['resultat'] = le_resultat.fit_transform(df['resultat'])
df['orientation'] = le_orientation.fit_transform(df['orientation'])
df['code_pays'] = le_code_pays.fit_transform(df['code_pays'])

# Sélectionner les features (variables indépendantes) et la cible (variable dépendante)
X = df[['specialite', 'mention', 'type_diplome', 'sexe', 'resultat', 'code_pays', 'moyenne_bac', 'score']]
y = df['orientation']  # La colonne à prédire

# 3. Séparer les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Entraînement du modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


# 5. Évaluation du modèle
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# 6. Test 1 : Faire une prédiction avec un premier ensemble de nouvelles données
new_data_1 = {
    'specialite': ['MATH'],
    'mention': ['Passable'],
    'type_diplome': ['bac'],
    'sexe': ['F'],
    'resultat': ['ADMIS(E)'],
    'code_pays': ['TN'],
    'moyenne_bac': [11.89],
    'score': [51.02]
}

# Convertir les nouvelles données en DataFrame et appliquer l'encodage
new_data_df_1 = pd.DataFrame(new_data_1)

# Vérifier que les valeurs de 'specialite', 'mention', 'type_diplome', etc., existent dans les données d'entraînement
new_data_df_1['specialite'] = le_specialite.transform(new_data_df_1['specialite'])
new_data_df_1['mention'] = le_mention.transform(new_data_df_1['mention'])
new_data_df_1['type_diplome'] = le_type_diplome.transform(new_data_df_1['type_diplome'])
new_data_df_1['sexe'] = le_sexe.transform(new_data_df_1['sexe'])
new_data_df_1['resultat'] = le_resultat.transform(new_data_df_1['resultat'])
new_data_df_1['code_pays'] = le_code_pays.transform(new_data_df_1['code_pays'])

# Faire une prédiction avec le modèle pour le premier test
predicted_orientation_1 = model.predict(new_data_df_1)

# Convertir la prédiction de l'orientation de nombre à étiquette (inverse de l'encodage)
predicted_orientation_label_1 = le_orientation.inverse_transform(predicted_orientation_1)

# Afficher le résultat du premier test
print(f"Test 1 - L'orientation prédite est : {predicted_orientation_label_1[0]}")

# 7. Test 2 : Faire une prédiction avec un deuxième ensemble de nouvelles données
new_data_2 = {
    'specialite': ['MATH'],
    'mention': ['Bien'],
    'type_diplome': ['bac'],
    'sexe': ['F'],
    'resultat': ['REFUSE(E)'],
    'code_pays': ['TN'],
    'moyenne_bac': [14.80],
    'score': [25.65]
}

# Convertir les nouvelles données en DataFrame et appliquer l'encodage
new_data_df_2 = pd.DataFrame(new_data_2)

# Vérifier que les valeurs de 'specialite', 'mention', 'type_diplome', etc., existent dans les données d'entraînement
new_data_df_2['specialite'] = le_specialite.transform(new_data_df_2['specialite'])
new_data_df_2['mention'] = le_mention.transform(new_data_df_2['mention'])
new_data_df_2['type_diplome'] = le_type_diplome.transform(new_data_df_2['type_diplome'])
new_data_df_2['sexe'] = le_sexe.transform(new_data_df_2['sexe'])
new_data_df_2['resultat'] = le_resultat.transform(new_data_df_2['resultat'])
new_data_df_2['code_pays'] = le_code_pays.transform(new_data_df_2['code_pays'])

# Faire une prédiction avec le modèle pour le deuxième test
predicted_orientation_2 = model.predict(new_data_df_2)

# Convertir la prédiction de l'orientation de nombre à étiquette (inverse de l'encodage)
predicted_orientation_label_2 = le_orientation.inverse_transform(predicted_orientation_2)

# Afficher le résultat du deuxième test
print(f"Test 2 - L'orientation prédite est : {predicted_orientation_label_2[0]}")

# 8. Test 3 : Faire une prédiction avec un troisième ensemble de nouvelles données
new_data_3 = {
    'specialite': ['MATH'],
    'mention': ['Assez Bien'],
    'type_diplome': ['bac'],
    'sexe': ['F'],
    'resultat': ['LISTE ATTENTE'],
    'code_pays': ['TN'],
    'moyenne_bac': [14.61],
    'score': [85.76]
}

# Convertir les nouvelles données en DataFrame et appliquer l'encodage
new_data_df_3 = pd.DataFrame(new_data_3)

# Vérifier que les valeurs de 'specialite', 'mention', 'type_diplome', etc., existent dans les données d'entraînement
new_data_df_3['specialite'] = le_specialite.transform(new_data_df_3['specialite'])
new_data_df_3['mention'] = le_mention.transform(new_data_df_3['mention'])
new_data_df_3['type_diplome'] = le_type_diplome.transform(new_data_df_3['type_diplome'])
new_data_df_3['sexe'] = le_sexe.transform(new_data_df_3['sexe'])
new_data_df_3['resultat'] = le_resultat.transform(new_data_df_3['resultat'])
new_data_df_3['code_pays'] = le_code_pays.transform(new_data_df_3['code_pays'])

# Faire une prédiction avec le modèle pour le troisième test
predicted_orientation_3 = model.predict(new_data_df_3)

# Convertir la prédiction de l'orientation de nombre à étiquette (inverse de l'encodage)
predicted_orientation_label_3 = le_orientation.inverse_transform(predicted_orientation_3)

# Afficher le résultat du troisième test
print(f"Test 3 - L'orientation prédite est : {predicted_orientation_label_3[0]}")


C:\Users\felah\AppData\Local\Temp\ipykernel_21620\3542989243.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Accuracy: 0.6573705179282868
Test 1 - L'orientation prédite est : INFO
Test 2 - L'orientation prédite est : TELCOMM
Test 3 - L'orientation prédite est : INFO
